## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

2025-05-18 17:08:09.536655: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:


# Create a list of at least 10 column names to use as X data
selected_features = [
    'Age',
    'DistanceFromHome',
    'Education',
    'EnvironmentSatisfaction',
    'JobInvolvement',
    'JobLevel',
    'JobSatisfaction',
    'WorkLifeBalance',
    'YearsAtCompany',
    'PerformanceRating'
]


# Create X_df using your selected columns
X_df = attrition_df[selected_features]


# Show the data types for X_df
X_df.dtypes



Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
WorkLifeBalance            int64
YearsAtCompany             int64
PerformanceRating          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y_df,
    test_size=0.3,
    random_state=42,
    stratify=y_df['Attrition']
)


In [6]:
# Convert X_train and X_test to float32 just to be safe for TensorFlow
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')



In [7]:

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create the encoder
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Transform the department column in train and test sets
y_train_dept = dept_encoder.transform(y_train[['Department']])
y_test_dept = dept_encoder.transform(y_test[['Department']])


## Part 2: Create, Compile, and Train the Model

In [9]:
from tensorflow.keras.layers import Input, Dense

# Find the number of columns in the X training data.
input_shape = X_train_scaled.shape[1]
print(f"Number of input features: {input_shape}")

# Create the input layer
input_layer = Input(shape=(input_shape,))

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

Number of input features: 10


In [10]:
# Create the hidden layer for Department branch
dept_hidden = Dense(16, activation='relu')(shared_layer_2)

# Output layer for Department (3 classes: assume OneHotEncoded)
dept_output = Dense(y_train_dept.shape[1], activation='softmax', name='department_output')(dept_hidden)



In [13]:
# Create the encoder
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder on the training data
attrition_encoder.fit(y_train[['Attrition']])

# Transform both training and testing data
y_train_attr = attrition_encoder.transform(y_train[['Attrition']])
y_test_attr = attrition_encoder.transform(y_test[['Attrition']])


In [15]:
# Create the hidden layer for Attrition branch
attr_hidden = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer for Attrition (2 classes: Yes/No, one-hot encoded)
attr_output = Dense(y_train_attr.shape[1], activation='softmax', name='attrition_output')(attr_hidden)

In [16]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_4[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_dept, 'attrition_output': y_train_attr},
    validation_data=(X_test_scaled, {'department_output': y_test_dept, 'attrition_output': y_test_attr}),
    epochs=50,
    batch_size=32
)


Epoch 1/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - attrition_output_accuracy: 0.8154 - attrition_output_loss: 0.5303 - department_output_accuracy: 0.5342 - department_output_loss: 0.9434 - loss: 1.4735 - val_attrition_output_accuracy: 0.8390 - val_attrition_output_loss: 0.4798 - val_department_output_accuracy: 0.6463 - val_department_output_loss: 0.8148 - val_loss: 1.2910
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8365 - attrition_output_loss: 0.4580 - department_output_accuracy: 0.6494 - department_output_loss: 0.7726 - loss: 1.2306 - val_attrition_output_accuracy: 0.8390 - val_attrition_output_loss: 0.4571 - val_department_output_accuracy: 0.6463 - val_department_output_loss: 0.7965 - val_loss: 1.2498
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8420 - attrition_output_loss: 0.3963 - department_output_accuracy: 0.6285 - department_output_loss: 0.7645 - loss: 1.1608 - val_attrition_output_accuracy: 0.8413 - va

In [23]:
# Evaluate the model with the testing data

model.evaluate(
    X_test_scaled,
    {'department_output': y_test_dept, 'attrition_output': y_test_attr},
    verbose=2
)

eval_results = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_dept, 'attrition_output': y_test_attr},
    verbose=0
)

# Show all outputs to see index positions
for i, metric in enumerate(eval_results):
    print(f"Metric {i}: {metric}")


14/14 - 0s - 6ms/step - attrition_output_accuracy: 0.8095 - attrition_output_loss: 0.5832 - department_output_accuracy: 0.6100 - department_output_loss: 0.9222 - loss: 1.4991
Metric 0: 1.4991438388824463
Metric 1: 0.9221720099449158
Metric 2: 0.5831690430641174
Metric 3: 0.8095238208770752
Metric 4: 0.6099773049354553


In [ ]:
# Evaluate and store results
eval_results = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_dept, 'attrition_output': y_test_attr},
    verbose=0
)

# Print the accuracy for both department and attrition
print(f"Attrition Accuracy: {eval_results[3]:.4f}")
print(f"Department Accuracy: {eval_results[4]:.4f}")



Department Accuracy: 0.6100


IndexError: list index out of range

Department Accuracy: 0.6100


IndexError: list index out of range

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 